In [100]:
import os
import tempfile

import keras
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.utils.np_utils import to_categorical

import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.stem import WordNetLemmatizer

import gensim
from gensim.models import Word2Vec

In [84]:
import sys
sys.setdefaultencoding('utf8')

In [19]:
df = pd.read_csv('train.csv')

In [58]:
X_train, y_train = df['text'], df['author']

In [59]:
len(X_train), len(y_train)

(19579, 19579)

## Embedding: generate vectors

In [108]:
def process(string_list):
    print(string_list)
    new_list = [clean(nltk.word_tokenize(s.replace('æ', 'ae'))) for s in string_list]
    return new_list

def clean(sentence, stop_words=nltk.corpus.stopwords.words('english'), lemm=WordNetLemmatizer()):
    return [lemm.lemmatize(x.lower()) for x in sentence if x.lower() not in stop_words]

In [109]:
clean(nltk.word_tokenize("I told you alreædy that the car looks nice"))

['told', 'alre\xc3\xa6dy', 'car', u'look', 'nice']

In [110]:
for x in X_train:
    print(x)
    process(x)
print("hop")

UnicodeDecodeError: 'utf8' codec can't decode byte 0xc3 in position 0: unexpected end of data

In [103]:
sentences = process([x for x in X_train])
vector_size = 50
model = Word2Vec(sentences, size=vector_size, window=10, min_count=1, workers=4)
word_vectors = model.wv
del model

UnicodeDecodeError: 'utf8' codec can't decode byte 0xc3 in position 2: unexpected end of data

In [87]:
def vectorize_word(word):
    try:
        return word_vectors[word.lower()]
    except:
        return word_vectors['the']
                            

def vectorize(text):
    return [vectorize_word(w) for w in text.split()]

X_train_vec = [vectorize(text) for text in X_train]

In [27]:
print len(y_train)
y_train = [{'EAP': 0, 'HPL': 1, 'MWS': 2}[y] for y in y_train]
print len(y_train)

19579
19579


In [28]:
X_train_vec = np.array(X_train_vec)
y_train = to_categorical(np.array(y_train), num_classes=3)

In [29]:
len(y_train)

19579

In [30]:
# truncate and pad input sequences
X_train = X_train_vec
max_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_length)

In [31]:
X_train.shape

(19579, 500, 50)

In [35]:
word_vectors.most_similar('bad')

[('"man', 0.9886143803596497),
 ('good', 0.985379159450531),
 ('acts', 0.9848403334617615),
 ('characteristic', 0.9842544794082642),
 ('alive', 0.9840017557144165),
 ('surely', 0.9824926853179932),
 ('pretty', 0.9819197058677673),
 ('fairly', 0.9813153147697449),
 ('sufficiently', 0.981155514717102),
 ('anybody', 0.9804400205612183)]

In [47]:
model = Sequential()
model.add(LSTM(128, input_shape=(500, vector_size)))
model.add(Dense(128, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(3, activation="softmax"))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=3, validation_split=0.33, batch_size=32)

Train on 13117 samples, validate on 6462 samples
Epoch 1/3
 6176/13117 [=============>................] - ETA: 475s - loss: 1.0129 - acc: 0.4817

KeyboardInterrupt: 

In [26]:
prediction = model.predict(np.array([X_train[3]]))
print prediction, y_train[3]

[[ 0.48820114  0.15095983  0.36083898]] [ 0.  0.  1.]


In [27]:
test_df = pd.read_csv('test.csv')
X_test_text = test_df['text']

In [28]:
X_test = [vectorize(text) for text in X_test_text]
X_test = sequence.pad_sequences(X_test, maxlen=max_length)

In [29]:
ids, y_test = [i for i in test_df['id']], model.predict(X_test)

In [30]:
# submission_file = open('submission1.csv', 'w')
file_text = ""
file_text += "id,EAP,HPL,MWS\n"
for data in zip(ids, y_test):
    file_text += "{},{},{},{}\n".format(data[0], data[1][0], data[1][1], data[1][2])
print(file_text[:200])

id,EAP,HPL,MWS
id02310,0.0450474508107,0.0615367554128,0.893415808678
id24541,0.969961106777,0.0161529406905,0.0138858770952
id00134,0.0378526486456,0.716230392456,0.245916977525
id27757,0.83968108892


In [31]:
submission_file = open('submission3.csv', 'w')
submission_file.write(file_text)
submission_file.close()

In [53]:
nltk.word_tokenize('I really love leticia evaristo!!')
from nltk.stem import WordNetLemmatizer
lemm = WordNetLemmatizer()
print("The lemmatized form of leaves is: {}".format(lemm.lemmatize("leaves")))

The lemmatized form of leaves is: leaf
